In [1]:
# TODO:
# problem: chefs club alias changed; ma peche has irregular characters
# unittest
# rating_better & rating_worse over time
# date_added, date_visited
# test PSQL locally; store as JSON

In [2]:
# TESTS:
# test that ratings_distribution_text is a list of length 10
# test that ratings_distribution data averages to rating

In [3]:
# date_visited_rating = ratings_history[date_visited.year][date_visited.month-1][1]
# most_recent_rating = ratings_history[max(ratings_history.keys())][-1][1]

In [4]:
# Imports
import ast
import bs4
import collections
import copy
import json
import pandas
import re
import requests
import sqlite3

### Define

In [5]:
def get_yelp_alias(soup):
    # TODO: test that input yelp_alias == yelp_alias; url may get re-routed
    yelp_alias = soup.find('meta', {'property': 'og:url'})
    yelp_alias = yelp_alias['content'].split('/')[-1]
    return yelp_alias

In [6]:
def get_yelp_biz_id(soup):
    yelp_biz_id = soup.find('meta', {'name': 'yelp-biz-id'})
    yelp_biz_id = yelp_biz_id['content']
    return yelp_biz_id

In [7]:
def get_is_closed(soup):
    is_closed = 'CLOSED' in soup.title.text
    return is_closed

In [8]:
def get_business_name(soup):
    business_name = soup.find('h1', {'class': 'biz-page-title'})
    business_name = business_name.text.strip()
    return business_name

In [9]:
def get_rating(soup):
    rating = soup.find('div', {'class': 'i-stars'})
    rating = rating['title'].split()[0]
    rating = float(rating)
    return rating

In [10]:
def get_review_count(soup):
    review_count = soup.find('span', {'class': 'review-count rating-qualifier'})
    review_count = review_count.text.strip().split()[0]
    review_count = int(review_count)
    return review_count

In [11]:
def get_ratings_history(soup):
    ratings_history = soup.find('div', {'id': 'rating-details-modal-content'})
    ratings_history = ratings_history['data-monthly-ratings']
    ratings_history = ast.literal_eval(ratings_history)
    ratings_history = {int(k):v for k,v in ratings_history.items()}
    ratings_history = json.dumps(ratings_history)    # stringify for sqlite; note that JSON keys are strings 
    return ratings_history

In [12]:
def get_ratings_distribution(soup):
    # Convert HTML table to len-10 string list
    text = soup.find('table', {'class': 'histogram'})
    text = text.text.replace('\n', ' ')
    text = re.sub(' {2,}', ',', text)    # Replace 2+ whitespace with 1 comma
    text = text.strip(',').split(',')
    
    # Convert string list to dictionary
    ratings = [text[i] for i in range(len(text)) if i%2 == 0]
    ratings = [i.split()[0] for i in ratings]
    ratings = [int(i) for i in ratings]
    counts  = [text[i] for i in range(len(text)) if i%2 != 0]
    counts  = [int(i) for i in counts]
    ratings_distribution = {'rating' : ratings, 'count' : counts}
    ratings_distribution = json.dumps(ratings_distribution)    # stringify for sqlite
    return ratings_distribution

In [13]:
# ratings_distribution = pandas.DataFrame(ratings_distribution)
# ratings_distribution['contrib'] = ratings_distribution['rating'] * ratings_distribution['count']
# ratings_distribution['contrib'].sum()/ratings_distribution['count'].sum()

In [14]:
def get_price_range(soup):
    price_range = soup.find('span', {'class': 'business-attribute price-range'})
    price_range = price_range.text.count('$')
    return price_range

In [15]:
def get_category_list(soup):
    category_list = soup.find('span', {'class': 'category-str-list'})
    category_list = category_list.contents
    category_list = [i.string for i in category_list]
    category_list = [i for i in category_list if '\n' not in i]
    category_list = json.dumps(category_list)    # stringify for sqlite
    return category_list

In [16]:
def get_address(soup):
    address = soup.find('address')
    address = copy.copy(address)    # Make a copy to prevent modifying the original document
    address.find('br').replace_with('\n')
    address = address.text.strip().split('\n')
    address = ', '.join(address)
    return address

In [17]:
def get_neighborhood_list(soup):
    neighborhood_list = soup.find('span', {'class': 'neighborhood-str-list'})
    if neighborhood_list is not None:
        neighborhood_list = neighborhood_list.text
        neighborhood_list = neighborhood_list.strip().split(', ')
        neighborhood_list = json.dumps(neighborhood_list)    # stringify for sqlite
    return neighborhood_list

### Testing

In [18]:
yelp_alias = 'momofuku-má-pêche-new-york-2'

In [19]:
url = 'https://www.yelp.com/biz/{}'.format(yelp_alias)
r = requests.get(url, timeout=5)
soup = bs4.BeautifulSoup(r.text, 'lxml')

In [20]:
yelp_alias = soup.find('meta', {'property': 'og:url'})
yelp_alias = yelp_alias['content'].split('/')[-1]
yelp_alias

'momofuku-m%C3%A1-p%C3%AAche-new-york-2'

In [21]:
price_range = soup.find('span', {'class': 'business-attribute price-range'})
price_range = price_range.text.count('$')
price_range

3

In [22]:
rating = soup.find('div', {'class': 'i-stars'})
rating = rating['title'].split()[0]
rating = float(rating)
rating

3.5

In [23]:
address = soup.find('address')
address = copy.copy(address)    # Make a copy to prevent modifying the original document
address.find('br').replace_with('\n')
address = address.text.strip().split('\n')
address = ', '.join(address)
address

'Chambers Hotel, 15 W 56th StNew York, NY 10019'

### Run

In [24]:
yelp_aliases = pandas.read_csv('yelp_aliases.csv')

In [25]:
yelp_aliases['date_added'] = yelp_aliases['date_added'].apply(lambda x: None if x == '-' else x)
yelp_aliases['date_added'] = pandas.to_datetime(yelp_aliases['date_added'])
yelp_aliases['date_visited'] = pandas.to_datetime(yelp_aliases['date_visited'])

In [26]:
# I did not add aliases for some food carts & bakeries; drop them for now
yelp_aliases = yelp_aliases[yelp_aliases['yelp_alias'].notna()]

In [27]:
# PROBLEM: chefs club alias changed; ma peche has irregular characters
# connection = sqlite3.connect('restaurants.db')
# already_done = pandas.read_sql_query('select * from restaurants', connection)
# already_done = already_done['yelp_alias'].tolist()
# yelp_aliases = yelp_aliases[yelp_aliases['yelp_alias'].apply(lambda x: x not in already_done)]

In [28]:
# Iterate
all_data = pandas.DataFrame()
for index, row in yelp_aliases.iterrows():
    # Set up soup
    yelp_alias = row['yelp_alias']
    url = 'https://www.yelp.com/biz/{}'.format(yelp_alias)
    r = requests.get(url, timeout=5)
    soup = bs4.BeautifulSoup(r.text, 'lxml')
    
    # Get data
    data = collections.OrderedDict()
    data['yelp_alias'] = get_yelp_alias(soup)
    data['yelp_biz_id'] = get_yelp_biz_id(soup)
    data['is_closed'] = get_is_closed(soup)
    data['business_name'] = get_business_name(soup)
    data['rating'] = get_rating(soup)
    data['review_count'] = get_review_count(soup)
    data['ratings_history'] = get_ratings_history(soup)
    data['price_range'] = get_price_range(soup)
    data['category_list'] = get_category_list(soup)
    data['address'] = get_address(soup)
    data['neighborhood_list'] = get_neighborhood_list(soup)
    
    # Append
    print([data['yelp_alias'], data['business_name']])
    data = pandas.Series(data).to_frame().T
    all_data = all_data.append(data, ignore_index=True)

['muk-eun-ji-new-york', 'Muk Eun Ji']
['ramenco-new-york-2', 'RamenCo']
['blt-bar-and-grill-new-york', 'BLT Bar & Grill']
['bills-bar-and-burger-downtown-new-york', 'Bill’s Bar & Burger Downtown']
['ruchi-new-york-2', 'Ruchi']
['komegashi-too-jersey-city', 'Komegashi Too']
['smashburger-new-york-6', 'Smashburger']
['harrys-italian-new-york-2', 'Harry’s Italian']
['goa-taco-new-york', 'goa taco']
['morgensterns-finest-ice-cream-new-york-2', 'Morgenstern’s Finest Ice Cream']
['vanessas-dumpling-house-new-york-2', 'Vanessa’s Dumpling House']
['philip-marie-new-york', 'Philip Marie']
['joes-shanghai-new-york-2', 'Joe’s Shanghai']
['oka-sushi-new-york', 'Oka Sushi']
['pisillo-italian-panini-new-york', 'Pisillo Italian Panini']
['friedmans-new-york-59', 'Friedman’s']
['prince-street-pizza-new-york-2', 'Prince Street Pizza']
['the-crooked-knife-new-york-3', 'The Crooked Knife']
['ramen-ya-new-york-5', 'Ramen-Ya']
['hamilton-pork-jersey-city', 'Hamilton Pork']
['the-malt-house-new-york-3', 'Th

In [29]:
# TODO: check data is valid before writing to DB
connection = sqlite3.connect('restaurants.db')
all_data.to_sql('restaurants', connection, if_exists='replace')

### Read DB Testing

In [30]:
data = pandas.read_sql_query('select * from restaurants', connection)

In [31]:
data.head()

,index,yelp_alias,yelp_biz_id,is_closed,business_name,rating,review_count,ratings_history,price_range,category_list,address,neighborhood_list
0,0,muk-eun-ji-new-york,g_F9WJJpRFB40oPJdoD2uA,1,Muk Eun Ji,3.5,365,"{""2016"": [[0, 4.0], [1, 3.5], [2, 3.0], [3, 3....",2,"[""Korean"", ""Barbeque""]","34 W 32nd St, Fl 1New York, NY 10001","[""Koreatown"", ""Midtown West""]"
1,1,ramenco-new-york-2,XBuzf_C9ewVerOQ0pEpjTA,1,RamenCo,3.0,229,"{""2016"": [[0, 2.5], [1, 2.5], [2, 3.0], [3, 3....",2,"[""Ramen""]","191 Pearl St, New York, NY 10038","[""Financial District""]"
2,2,blt-bar-and-grill-new-york,52cxQ_pGIEvdXQcRa9GfUA,1,BLT Bar & Grill,3.0,163,"{""2016"": [[0, 4.0], [1, 4.0], [2, 4.0], [3, 4....",2,"[""American (Traditional)""]","123 Washington St, New York, NY 10006","[""Financial District""]"
3,3,bills-bar-and-burger-downtown-new-york,fvYQeoC42TkIyPTSVEQWvw,0,Bill’s Bar & Burger Downtown,3.5,420,"{""2016"": [[0, 3.0], [1, 3.0], [2, 3.0], [3, 3....",2,"[""Bars"", ""Burgers""]","85 West St, New York, NY 10006","[""Financial District""]"
4,4,ruchi-new-york-2,HGNqWrRYJNyhJSS8yia8HQ,0,Ruchi,3.5,310,"{""2016"": [[0, 3.0], [1, 3.0], [2, 3.0], [3, 3....",2,"[""Indian""]","120 Cedar St, New York, NY 10006","[""Financial District""]"


In [32]:
data['ratings_history'] = data['ratings_history'].apply(ast.literal_eval)

In [33]:
data['ratings_history'][0].keys()

dict_keys(['2016', '2017', '2018', '2014', '2015'])

In [34]:
connection = sqlite3.connect('restaurants.db')
pandas.read_sql_query('select * from sqlite_master', connection)

,type,name,tbl_name,rootpage,sql
0,table,restaurants,restaurants,2,"CREATE TABLE ""restaurants"" (\n""index"" INTEGER,..."
1,index,ix_restaurants_index,restaurants,4,"CREATE INDEX ""ix_restaurants_index""ON ""restaur..."
